In [ ]:
import visa                             # Import PyVISA library
import time
from struct import *                    # Import Struct module (interpret strings as packed binary data)
import numpy as np
#import VISAresourceExtentions

In [ ]:
# Initialization

rm = visa.ResourceManager()             # Create Resource Manager object
rs = rm.list_resources()                # Method to list the available resources
print(rs[0])                            
counter = rm.open_resource(rs[0])       # Assign the returned object to the instrument variable (i.e. counter)

In [ ]:
# Initial settings

print(counter.query('*IDN?'))           # Query the Identification string 
counter.write('*RST;*CLS')              # Reset the instrument, clear the Error queue
counter.timeout = 1e7                   # Acquisition timeout (ms) - set it higher than the acquisition time
counter.query('SYST:ERR?')              # Error Checking 

In [ ]:
#Basic settings

counter.write("FUNC 'FREQ:BTB 1'")                      # Setup for frequency back-to-back measurement from channel A (1) - Set up for period back-to-back is "FUNC 'PER:BTB 1'" 
counter.write('CALC:AVER:STAT ON')                      # Enable/disable statitics;
counter.write('INP:LEV:AUTO OFF; :INP:LEV 0')           # Enable/Disable autotrigger on channel A; Trigger level (V)
counter.write('CAL:INT:AUTO OFF; :DISP:ENAB ON')        # Enable/Disable reciprocal counter that uses an interpolating technique to increase the resolution; Enable/Disable the display (if OFF increase the GPIB speed)
counter.query('SYST:ERR?')                              # Error Checking    

In [ ]:
#counter.write('*ESE 0;*SRE 0')                      # To be developed
#counter.wait_on_event(counter, RQS, 10000)
#counter.last_status
#counter.query('*ESR?')
#counter.read()
#counter.query('SYST:ERR?')

In [ ]:
# Calculate MEAN/MAX/MIN/SDEV/ADEV values (ascii format), only if 'CALC:AVER:STAT ON'

counter.write('CALC:AVER:TYPE MEAN')                                      # Select the statistical function to be performed (MIN/MAX/MEAN/SDEV/ADEV)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')                            # Read timestamp of each measurement ; Redout in ASCII/REAL mode - Redout format: ASCII/REAL -> [freq (Hz), tstamp (sec)]
counter.write('SENSE:ACQ:APER 4e-6; :CALC:AVER:COUNT 1e3')                # Gate time (sec) - (In statistics mode is the Pacing time of the samples), minimum: 4 microsec; Set the number of samples to use in statistics sampling                                      
#time.sleep(1)                                                             # Wait (sec)                                                              

start = time.time()

for i in range(1):
    
    counter.query('INIT;*OPC?')                                           # Initialize acquisition; *OPC? query waits until the acquisition ends

    #counter.write('CALC:DATA?')                                 
    #counter.read()
    #value = counter.query('CALC:DATA?')                                   # Fetch list with string characters as its elements
    #value = counter.query_ascii_values('CALC:DATA?')                      # Fetch ASCII value ( ValueError: 'could not convert string to float can occur' ), sometimes doesn't work when timestamp is printed (i.e. with MIN/MAX )         
    value = counter.query_ascii_values('CALC:DATA?', converter = 's')     # Fetch and Convert ASCII value into a string
    
    value[-1] = value[-1].split('\n')[0]                                  # Delete in the string every character after the number ( i.e. \n0E..)
    value[0] = float(value[0])                                            # Convert string into float number
    value[1] = float(value[1])                                            # Condition necessary when the last element of the list happens to be an empty string = '' 
    #print(value)                                                         # List format -> [value, timestamp]
    
    freqmean = value[0]
    print(freqmean)
    
end = time.time()
print(end-start)                                                           # Total acquisition time (sec)
counter.query('SYST:ERR?')    

In [ ]:
# Calculate ADEV for different Gate times

counter.write('CALC:AVER:TYPE ADEV')                                      # Select the statistical function to be performed (MIN/MAX/MEAN/SDEV/ADEV)
counter.write('FORMAT:TINF ON; :FORMAT ASCII')                            # Read timestamp of each measurement ; Redout in ASCII/REAL mode - Redout format: ASCII/REAL -> [freq (Hz), tstamp (sec)]
#time.sleep(1)                                                             # Wait (sec)                                                                

start = time.time()

Taulist = []       # Define list: list=[] ; Define numpy array:  arr=np.array([])
Adevlist = [] 
Sampleslist = []

Samp=[50000, 25000, 16667, 10000, 6250, 4000, 2565, 1588, 1011, 637, 402, 254, 160, 101, 64, 41, 26, 16, 11, 7]    # Define list of samples to take for each gate time

for i in range(4):
    
    x = i-5  # [-5,-4,-3,-2,-1,0,+1]
    
    for y in [0,1,2,3,4]:                                       # Iterations - NotEvenlySpaced: [1,2,4,6,8] - EvenlySpaced: [0,1,2,3,4]
        
        tau = (10**x)*10**(y/5)                                 # Gate time (sec) - NotEvenlySpaced: y*10**(x) - EvenlySpaced: (10**x)*10**(y/5)            
        
        samples = Samp[5*i+y]                                   # Number of Samples:  10**abs(x) - 1000 - ...
        #samples = {x==-1: 100, x==0: 10, x==1: 2}.get(True)     # Instead of if(): ... elif(): ... elif() ... 
        
        #if (x == -5 and y == 2):   break                        # Break the loop when condition is fulfilled
        
        print(f'Gate time = {tau} s, Samples = {samples}')            
    
        counter.write(f'SENSE:ACQ:APER {tau}; :CALC:AVER:COUNT {samples}')    # Set Gate time (sec) and number of Samples 
        counter.query('INIT;*OPC?')                                           # Initialize acquisition; *OPC? query waits until the acquisition ends 
        
        #value = counter.query_ascii_values('CALC:DATA?')                     # Fetch and Convert ASCII value into a string
        value = counter.query_ascii_values('CALC:DATA?', converter = 's')     # Fetch and Convert ASCII value into a string

        value[-1] = value[-1].split('\n')[0]                                  # Delete in the string every character after the number ( i.e. \n0E..)
        value[0] = float(value[0])                                            # Convert string into float number
        value[1] = float(value[1])
        print(value[0])
        
        Taulist.append(tau)                                                   # Append a value to the list at each iteration
        Adevlist.append(value[0])
        Sampleslist.append(samples)
         
        #end = time.time()     
        #print(end-start)                                                      # Acquisition time for each Gate time (sec)

end = time.time()
print(end-start)                                                              # Total acquisition time (sec)
            
counter.query('SYST:ERR?')  

In [ ]:
print(Taulist)  
print(Adevlist)
print(Sampleslist)

In [ ]:
# ADEV vs Gate time plot

import matplotlib.pyplot as plt
import pandas as pd

# Convert lists to numpy arrays

TauArray = np.array(Taulist)      
AdevArray = np.array(Adevlist)
SamplesArray = np.array(Sampleslist)

print(repr(SamplesArray))
print(repr(TauArray))
print(repr(AdevArray))

ADEVArray = AdevArray/freqmean   # Calculate normalized Adev

print(repr(ADEVArray))

plt.plot(TauArray, ADEVArray, 'o-')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.show()

In [ ]:
#EvenlySpaced

Eadev1=ADEVArray
Etau1=TauArray

In [ ]:
Eadev2=ADEVArray

In [ ]:
Eadev3=ADEVArray

In [ ]:
Eadev4=ADEVArray

In [ ]:
Eadev5=ADEVArray

In [ ]:
#NotEvenlySpaced

NEadev1=ADEVArray
NEtau1=TauArray

In [ ]:
NEadev2=ADEVArray

In [ ]:
NEadev3=ADEVArray

In [ ]:
NEadev4=ADEVArray

In [ ]:
NEadev5=ADEVArray

In [ ]:
# Convert [1.4 2.5 6.7] -> [1.4, 2.5, 6.7]
arr = list(map(float, input().split()))
arr = np.array(arr)

In [ ]:
print(repr(Etau1))
print(repr(NEtau1))

In [ ]:
print(repr(Eadev1))
print(repr(Eadev2))
print(repr(Eadev3))
print(repr(Eadev4))
print(repr(Eadev5))

In [ ]:
print(repr(NEadev1))
print(repr(NEadev2))
print(repr(NEadev3))
print(repr(NEadev4))
print(repr(NEadev5))

In [ ]:
#EvenlySpaced

plt.plot(Etau1, Eadev1, 'o-')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.show()

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd

In [ ]:
#EvenlySpaced
#Superposed plots

plt.figure(figsize=(12,8))

plt.plot(Etau1, Eadev1, 'o--', linewidth=1, markersize=5, label='1')      #2=3=4 - 5=6=7=8=9 - 
plt.plot(Etau1, Eadev2, 'o--', linewidth=1, markersize=5, label='2')
plt.plot(Etau1, Eadev3, 'o--', linewidth=1, markersize=5, label='3')
plt.plot(Etau1, Eadev4, 'o--', linewidth=1, markersize=5, label='4')
plt.plot(Etau1, Eadev5, 'o--', linewidth=1, markersize=5, label='5')


plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.legend()
plt.show()

In [ ]:
#NotEvenlySpaced

plt.plot(NEtau1, NEadev1, 'o-')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.show()

In [ ]:
#NotEvenlySpaced
#Superposed plots

plt.figure(figsize=(12,8))

plt.plot(NEtau1, NEadev1, 'o--', linewidth=1, markersize=5, label='1')
plt.plot(NEtau1, NEadev2, 'o--', linewidth=1, markersize=5, label='2')
plt.plot(NEtau1, NEadev3, 'o--', linewidth=1, markersize=5, label='3')
plt.plot(NEtau1, NEadev4, 'o--', linewidth=1, markersize=5, label='4')
plt.plot(NEtau1, NEadev5, 'o--', linewidth=1, markersize=5, label='5')

plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.legend()
plt.show()

In [ ]:
#Superposed plots
#EvenlySpaced vs Not EvenlySpaced

plt.plot(Etau1, Eadev1, 'o--', label='evenly spaced')
plt.plot(NEtau1, NEadev1, 'o--', label='not evenly spaced')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.legend()
plt.show()

In [ ]:
#Superposed plots
#EvenlySpaced vs Not EvenlySpaced

plt.figure(figsize=(12,8))

plt.plot(Etau1, Eadev1, 'o--', linewidth=1, markersize=4, label='evenly spaced', color='b')
plt.plot(Etau1, Eadev2, 'o--', linewidth=1, markersize=4, color='b')
plt.plot(Etau1, Eadev3, 'o--', linewidth=1, markersize=4, color='b')
plt.plot(Etau1, Eadev4, 'o--', linewidth=1, markersize=4, color='b')
plt.plot(Etau1, Eadev5, 'o--', linewidth=1, markersize=4, color='b')
plt.plot(NEtau1, NEadev1, 'o--', linewidth=1, markersize=4, label='not evenly spaced', color='r')
plt.plot(NEtau1, NEadev2, 'o--', linewidth=1, markersize=4, color='r')
plt.plot(NEtau1, NEadev3, 'o--', linewidth=1, markersize=4, color='r')
plt.plot(NEtau1, NEadev4, 'o--', linewidth=1, markersize=4, color='r')
plt.plot(NEtau1, NEadev5, 'o--', linewidth=1, markersize=4, color='r')
plt.xscale('log')
plt.yscale('log')
#plt.ticklabel_format(axis="y", style="sci", scilimits=(0,0))
plt.title('ADEV vs Gate time')
#plt.ylabel('Hz')
plt.xlabel('sec')
plt.legend()
plt.show()